Detección de caras con webcam

In [1]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors
import numpy as np
import math

En el bucle de procesamiento, las teclas 'd' y 'e' `permiten respectivamenet cambiar de modelo de detección de caras, y en su caso de máscara de detección del rostro.

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [8]:
glasses = cv2.imread("./images/glasses.png", cv2.IMREAD_UNCHANGED)
hat = cv2.imread("./images/sombrero.png", cv2.IMREAD_UNCHANGED)
mustache = cv2.imread("./images/mustache.png", cv2.IMREAD_UNCHANGED)

if glasses is not None:
    print("Dimensiones iniciales de las gafas:", glasses.shape)
else:
    print("No se pudo cargar la imagen de las gafas.")

if hat is not None:
    print("Dimensiones iniciales del sombrero:", hat.shape)
else:
    print("No se pudo cargar la imagen del sombrero.")

if mustache is not None:
    print("Dimensiones iniciales del bigote:", mustache.shape)
else:
    print("No se pudo cargar la imagen del bigote.")

normalizatorHS = faceutils.Normalization()

# -- Face detectors interface --
FDet = FaceDetectors.FaceDetector()

# -- Fonts --
font = cv2.FONT_HERSHEY_SIMPLEX

# -- Superpone una imagen de 'overlay' en una imagen de fondo en las coordenadas especificadas --
def overlay_image(im1, im2, x_offset, y_offset):
    try:
        y1, y2 = y_offset, y_offset + im2.shape[0]
        x1, x2 = x_offset, x_offset + im2.shape[1]

        alpha_s = im2[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s

        for c in range(3):
            im1[y1:y2, x1:x2, c] = (alpha_s * im2[:, :, c] + alpha_l * im1[y1:y2, x1:x2, c])

        return im1
    except:
        return im1
    
# Define a function to rotate an image by a given angle
def rotate_image(image, angle):
    center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return rotated_image

# -- Webcam connection --
cap = cv2.VideoCapture(0)
# -- Check for other cameras --
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

# -- Face detection and eye model setup --
imodoF = 0
imodoE = 0

debug = 0

# -- Set camera resolution --
cap.set(3,640);
cap.set(4,480);

while True:
    # -- Get frame --
    t = time.time()
    ret, frame = cap.read()
    # -- For HS normalization --
    B, G, R = cv2.split(frame)

    # -- Search face with a specific setup for face and eye detection --
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values

        # -- Draws face container-- 
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # -- Draws eyes and mask if available --
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # -- Show detected facial elements --
                if imodoF > 0:
                    for (x, y) in shape:
                        cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)

                # -- Normalize and show --
                # -- Color channels --
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
                cv2.imshow("Normalized", NormBGR)

                # -- Calculamos el factor de escala basado en el tamaño del cuadrado de la cara --
                scale_factor = w / 175

                # -- Redimensionamos las imágenes --
                hat_resized = cv2.resize(hat, None, fx=scale_factor, fy=scale_factor)
                mustache_resized = cv2.resize(mustache, None, fx=scale_factor, fy=scale_factor)
                glasses_resized = cv2.resize(glasses, None, fx=scale_factor, fy=scale_factor)

                # -- Calculate the middle of the eyes along the X-axis --
                middle_of_eyes_x = (lex + rex) // 2

                # -- Calculate the position of the mustache around the height of the nose --
                nose_height = min(ley, rey) + (rey - ley) // 2 

                # -- Calculate the position of the glasses based on the middle of the eyes --
                glasses_x = middle_of_eyes_x - (glasses_resized.shape[1] // 2)
                glasses_y = nose_height - 20

                glasses_x = max(0, glasses_x)
                glasses_y = max(0, glasses_y)

                # -- Calculate the position of the hat in the middle of the eyes and move it upwards --
                hat_x = middle_of_eyes_x - (hat_resized.shape[1] // 2)
                hat_y = min(ley, rey) - hat_resized.shape[0] - 30 

                hat_x = max(0, hat_x)
                hat_y = max(0, hat_y)


                # -- Calculate the position of the mustache based on the eyes --
                mustache_x = middle_of_eyes_x - (mustache_resized.shape[1] // 2)
                mustache_y = nose_height + 30  

                mustache_x = max(0, mustache_x)
                mustache_y = max(0, mustache_y)

                eye_angle = (np.degrees(np.arctan2(rey - ley, rex - lex))) * -1

                # Rotate the accessories based on the eye angle
                glasses_resized_rotated = rotate_image(glasses_resized, eye_angle)
                hat_resized_rotated = rotate_image(hat_resized, eye_angle)
                mustache_resized_rotated = rotate_image(mustache_resized, eye_angle)

                # Overlay the rotated accessories on the frame
                frame = overlay_image(frame, mustache_resized_rotated, mustache_x, mustache_y)
                frame = overlay_image(frame, hat_resized_rotated, hat_x, hat_y)
                frame = overlay_image(frame, glasses_resized_rotated, glasses_x, glasses_y)

    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))

    # -- Show resulting image --
    cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    if imodoF == 1 or imodoF == 2:
        cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Cam', frame)

    # -- Esc to finish --
    tec = cv2.waitKey(40)
    if tec & tec == 27: 
        break
    # -- Face detector change --
    elif tec & 0xFF == ord('d'):
        imodoF = imodoF + 1
        if imodoF >= len(FDet.FaceDetectors):
            imodoF = 0
    # -- Eye detector change --
    elif tec & 0xFF == ord('e'):
        imodoE = imodoE + 1
        if imodoE >= len(FDet.EyeDetectors):
            imodoE = 0

# -- Close windows and release camera --
cap.release()
cv2.destroyAllWindows()

Dimensiones iniciales de las gafas: (66, 159, 4)
Dimensiones iniciales del sombrero: (116, 160, 4)
Dimensiones iniciales del bigote: (50, 139, 4)
Camera 0
